### Step 1: Load Pre-trained models

In [3]:
import os
import sys
import torch
import torchvision.transforms as transforms
from PIL import Image
from tqdm.notebook import tqdm

# Add the path to the `pytorch-CycleGAN-and-pix2pix` repository
repo_path = '/Users/ls/Sites/pytorch-CycleGAN-and-pix2pix'  # Update this to your repository path
sys.path.append(repo_path)

from models.networks import define_G

# Function to load the generator model from a .pth file
def load_generator(model_path):
    # Define the generator architecture
    netG = define_G(input_nc=3, output_nc=3, ngf=64, netG='unet_256', norm='batch', use_dropout=False, init_type='normal', init_gain=0.02, gpu_ids=[])
    
    # Load the generator weights
    netG.load_state_dict(torch.load(model_path))
    netG.eval()
    return netG

# Paths to your generator .pth files
model1_path = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/ma-boston-p2p-500-150-v100/500_net_G.pth'
model2_path = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/nc-charlotte-500-150-v100/500_net_G.pth'
model3_path = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/ny-manhattan-p2p-500-150-v100/500_net_G.pth'
model4_path = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/pa-pittsburgh-p2p-500-150-v100/500_net_G.pth'

# Load models
generator1 = load_generator(model1_path)
generator2 = load_generator(model2_path)
generator3 = load_generator(model3_path)
generator4 = load_generator(model4_path)

# Load and preprocess your input dataset
input_dir = '/path/to/input/images'  # Update this to your input images path
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

def load_images(input_dir):
    images = []
    for filename in tqdm(sorted(os.listdir(input_dir)), desc="Loading images"):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        images.append(img)
    return torch.stack(images)

input_images = load_images(input_dir)

# Generate outputs from each generator
with torch.no_grad():
    outputs1 = generator1(input_images)
    outputs2 = generator2(input_images)
    outputs3 = generator3(input_images)
    outputs4 = generator4(input_images)

# Example: Combine outputs by averaging (simple ensemble method)
final_output = (outputs1 + outputs2 + outputs3 + outputs4) / 4

# Save or use the outputs as needed
# Example: Convert tensor to image and save
def save_image(tensor, path):
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = transforms.ToPILImage()(image)
    image.save(path)

output_dir = '/path/to/save/outputs'  # Update this to your output directory path
os.makedirs(output_dir, exist_ok=True)

for i, output in enumerate(final_output):
    save_image(output, os.path.join(output_dir, f'output_{i}.png'))

initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/input/images'

### Step 2: Generate outputs

In [ ]:
import torchvision.transforms as transforms
from PIL import Image
import os

# Load and preprocess your input dataset
input_dir = '/path/to/input/images'
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

def load_images(input_dir):
    images = []
    for filename in sorted(os.listdir(input_dir)):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        images.append(img)
    return torch.stack(images)

input_images = load_images(input_dir)

# Generate outputs from each generator
with torch.no_grad():
    outputs1 = generator1(input_images)
    outputs2 = generator2(input_images)
    outputs3 = generator3(input_images)
    outputs4 = generator4(input_images)

# Save or use the outputs as needed
# Example: Convert tensor to image and save
def save_image(tensor, path):
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = transforms.ToPILImage()(image)
    image.save(path)

output_dir = '/path/to/save/outputs'
os.makedirs(output_dir, exist_ok=True)

for i, output in enumerate(outputs1):
    save_image(output, os.path.join(output_dir, f'output_model1_{i}.png'))

for i, output in enumerate(outputs2):
    save_image(output, os.path.join(output_dir, f'output_model2_{i}.png'))

for i, output in enumerate(outputs3):
    save_image(output, os.path.join(output_dir, f'output_model3_{i}.png'))

for i, output in enumerate(outputs4):
    save_image(output, os.path.join(output_dir, f'output_model4_{i}.png'))

### Step 3: Combine Outputs Using Softmax Classifier

In [ ]:
import torch.nn.functional as F

# Combine outputs into a single tensor
combined_outputs = torch.stack([outputs1, outputs2, outputs3, outputs4], dim=1)

# Assuming you have a pre-trained softmax classifier
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super(SoftmaxClassifier, self).__init__()
        self.fc = nn.Linear(4, 4)  # Assuming 4 models

    def forward(self, x):
        x = self.fc(x)
        return F.softmax(x, dim=1)

softmax_classifier = SoftmaxClassifier()
softmax_classifier.load_state_dict(torch.load('/path/to/softmax_classifier.pth'))
softmax_classifier.eval()

# Generate softmax weights
weights = softmax_classifier(combined_outputs)

# Combine model outputs using the weights
final_output = torch.sum(weights.unsqueeze(2) * combined_outputs, dim=1)